# హోటల్ బుకింగ్ ప్రాధాన్యత సభ్యుల మిడిల్‌వేర్‌తో

ఈ నోట్‌బుక్ **ఫంక్షన్-ఆధారిత మిడిల్‌వేర్** ను Microsoft Agent Framework ఉపయోగించి ప్రదర్శిస్తుంది. మేము కండీషనల్ వర్క్‌ఫ్లో ఉదాహరణను తీసుకుని, ప్రాధాన్యత సభ్యులకు ప్రత్యేకమైన ప్రయోజనాలను అందించే మిడిల్‌వేర్ లేయర్‌ను జోడిస్తాము.

## మీరు నేర్చుకునే విషయాలు:
1. **ఫంక్షన్-ఆధారిత మిడిల్‌వేర్**: ఫంక్షన్ ఫలితాలను అడ్డుకోవడం మరియు మార్చడం
2. **కాంటెక్స్ట్ యాక్సెస్**: `context.result` ను అమలు తర్వాత చదవడం మరియు మార్చడం
3. **బిజినెస్ లాజిక్ అమలు**: ప్రాధాన్యత సభ్యుల ప్రయోజనాలు
4. **ఫలితాల ఓవర్‌రైడ్**: వినియోగదారు స్థితి ఆధారంగా ఫంక్షన్ ఫలితాలను మార్చడం
5. **అదే వర్క్‌ఫ్లో, వేర్వేరు ఫలితాలు**: మిడిల్‌వేర్-చలిత ప్రవర్తన మార్పులు

## మిడిల్‌వేర్‌తో వర్క్‌ఫ్లో ఆర్కిటెక్చర్:

```
User Input: "I want to book a hotel in Paris"
                    ↓
        [availability_agent]
        - Calls hotel_booking tool
        - 🌟 priority_check middleware intercepts
        - Checks user membership status
        - IF priority + no rooms → Override to available!
        - Returns BookingCheckResult
                    ↓
        Conditional Routing
           /                    \
    [has_availability]    [no_availability]
          ↓                      ↓
    [booking_agent]        [alternative_agent]
    (Priority override!)   (Regular users)
          ↓                      ↓
       [display_result executor]
```

## కండీషనల్ వర్క్‌ఫ్లోతో తేడా:

**మిడిల్‌వేర్ లేకుండా** (14-conditional-workflow.ipynb):
- పారిస్‌లో గదులు లేవు → alternative_agent కు రూట్ చేయబడుతుంది

**మిడిల్‌వేర్‌తో** (ఈ నోట్‌బుక్):
- సాధారణ వినియోగదారు + పారిస్ → గదులు లేవు → alternative_agent కు రూట్ చేయబడుతుంది
- ప్రాధాన్యత వినియోగదారు + పారిస్ → 🌟 మిడిల్‌వేర్ ఓవర్‌రైడ్! → అందుబాటులో ఉంది → booking_agent కు రూట్ చేయబడుతుంది

## అవసరమైనవి:
- Microsoft Agent Framework ఇన్‌స్టాల్ చేయబడింది
- కండీషనల్ వర్క్‌ఫ్లోల అవగాహన (14-conditional-workflow.ipynb చూడండి)
- GitHub టోకెన్ లేదా OpenAI API కీ
- మిడిల్‌వేర్ ప్యాటర్న్‌ల ప్రాథమిక అవగాహన


In [2]:
import asyncio
import json
import os
from collections.abc import Awaitable, Callable
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    FunctionInvocationContext,
    Role,
    WorkflowBuilder,
    WorkflowContext,
    ai_function,
    executor,
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")

✅ All imports successful!


## దశ 1: నిర్మిత అవుట్‌పుట్‌ల కోసం Pydantic మోడళ్లను నిర్వచించండి

ఈ మోడళ్లు ఏజెంట్లు తిరిగి పంపే **schema** ను నిర్వచిస్తాయి. మేము `priority_override` అనే ఫీల్డ్‌ను జోడించాము, ఇది మిడిల్‌వేర్ లభ్యత ఫలితాన్ని మార్చినప్పుడు ట్రాక్ చేయడానికి ఉపయోగపడుతుంది.


In [3]:
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""

    destination: str
    has_availability: bool
    message: str
    priority_override: bool = False  # 🆕 NEW! Tracks if middleware overrode the result


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""

    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""

    destination: str
    action: str
    message: str


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check with priority_override)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")

✅ Pydantic models defined:
   - BookingCheckResult (availability check with priority_override)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)


## దశ 2: ప్రాధాన్యత సభ్యుల డేటాబేస్‌ను నిర్వచించండి

ఈ డెమో కోసం, మేము ప్రాధాన్యత సభ్యత్వ డేటాబేస్‌ను అనుకరిస్తాము. ఉత్పత్తిలో, ఇది నిజమైన డేటాబేస్ లేదా APIని ప్రశ్నిస్తుంది.

**ప్రాధాన్యత సభ్యులు:**
- `alice@example.com` - VIP సభ్యుడు
- `bob@example.com` - ప్రీమియం సభ్యుడు  
- `priority_user` - పరీక్ష ఖాతా


In [4]:
# Simulated priority members database
PRIORITY_MEMBERS = {
    "alice@example.com",
    "bob@example.com",
    "priority_user",
}

# Global variable to track current user (in real app, use proper session management)
current_user_id = "regular_user"  # Default: regular user


def set_user(user_id: str):
    """Set the current user for the session."""
    global current_user_id
    current_user_id = user_id
    is_priority = user_id in PRIORITY_MEMBERS
    status = "🌟 PRIORITY MEMBER" if is_priority else "👤 Regular User"

    display(
        HTML(f"""
        <div style='padding: 15px; background: {"linear-gradient(135deg, #FFD700 0%, #FFA500 100%)" if is_priority else "#e3f2fd"}; 
                    border-left: 4px solid {"#FF6B35" if is_priority else "#2196f3"}; border-radius: 4px; margin: 10px 0;'>
            <strong>👤 Current User Set:</strong> {user_id}<br>
            <strong>Status:</strong> {status}
        </div>
    """)
    )


print("✅ Priority members database created")
print(f"   Priority members: {len(PRIORITY_MEMBERS)} users")

✅ Priority members database created
   Priority members: 3 users


## దశ 3: హోటల్ బుకింగ్ టూల్ సృష్టించండి

కండిషనల్ వర్క్‌ఫ్లోలాగే ఉంటుంది, కానీ ఇప్పుడు ఇది మిడిల్‌వేర్ ద్వారా ఆపబడుతుంది!


In [5]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## దశ 4: 🌟 ప్రాధాన్యత తనిఖీ మిడిల్‌వేర్ సృష్టించండి (ముఖ్యమైన ఫీచర్!)

ఇది ఈ నోట్‌బుక్ యొక్క **మూల ఫంక్షనాలిటీ**. మిడిల్‌వేర్:

1. హోటల్_బుకింగ్ ఫంక్షన్ కాల్‌ను **ఇంటర్‌సెప్ట్** చేస్తుంది  
2. `next(context)` ను పిలిచి ఫంక్షన్‌ను సాధారణంగా **నిర్వహిస్తుంది**  
3. `context.result` లో ఫలితాన్ని **పరిశీలిస్తుంది**  
4. వినియోగదారు ప్రాధాన్యత కలిగి ఉంటే మరియు గదులు అందుబాటులో లేకపోతే ఫలితాన్ని **ఓవర్‌రైడ్** చేస్తుంది  
5. **మార్చిన ఫలితాన్ని** ఏజెంట్‌కు తిరిగి ఇస్తుంది  

**ముఖ్యమైన నమూనా:**  
```python
async def my_middleware(context, next):
    await next(context)  # ఫంక్షన్‌ను అమలు చేయండి
    # ఇప్పుడు context.result ఫంక్షన్ యొక్క అవుట్‌పుట్‌ను కలిగి ఉంటుంది
    if some_condition:
        context.result = new_value  # ఓవర్‌రైడ్!
```


In [6]:
async def priority_check_middleware(
    context: FunctionInvocationContext,
    next: Callable[[FunctionInvocationContext], Awaitable[None]],
) -> None:
    """
    Function middleware that overrides hotel_booking results for priority members.
    
    Workflow:
    1. Let the function execute normally
    2. Check if user is a priority member
    3. If priority + no availability → Override to make rooms available!
    4. Agent will then route to booking path instead of alternative path
    """
    function_name = context.function.name

    display(
        HTML(f"""
        <div style='padding: 12px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
            <strong>🔄 Middleware:</strong> Intercepting {function_name}...
        </div>
    """)
    )

    # Execute the original function
    await next(context)

    # Now inspect and potentially modify the result
    if context.result and function_name == "hotel_booking":
        result_data = json.loads(context.result)
        destination = result_data.get("destination", "")
        has_availability = result_data.get("has_availability", False)

        # Check if user is priority member
        is_priority = current_user_id in PRIORITY_MEMBERS

        # Override logic: Priority member + no availability → Make available!
        if is_priority and not has_availability:
            display(
                HTML(f"""
                <div style='padding: 20px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); 
                            border-radius: 8px; margin: 10px 0; box-shadow: 0 4px 12px rgba(255,165,0,0.4);'>
                    <h3 style='margin: 0 0 10px 0; color: #333;'>🌟 PRIORITY OVERRIDE ACTIVATED! 🌟</h3>
                    <p style='margin: 0; color: #555; line-height: 1.6;'>
                        <strong>User:</strong> {current_user_id}<br>
                        <strong>Status:</strong> VIP Priority Member<br>
                        <strong>Action:</strong> Overriding "No Availability" for {destination}<br>
                        <strong>Result:</strong> ✅ Rooms now available for priority booking!
                    </p>
                </div>
            """)
            )

            # Override the result!
            result_data["has_availability"] = True
            result_data["priority_override"] = True
            context.result = json.dumps(result_data)

        elif not has_availability:
            display(
                HTML(f"""
                <div style='padding: 12px; background: #ffebee; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                    <strong>ℹ️ Middleware:</strong> No priority override (user: {current_user_id})
                </div>
            """)
            )


print("✅ priority_check_middleware created")
print("   - Intercepts hotel_booking function")
print("   - Overrides availability for priority members")

✅ priority_check_middleware created
   - Intercepts hotel_booking function
   - Overrides availability for priority members


## దశ 5: రూటింగ్ కోసం షరతు ఫంక్షన్లను నిర్వచించండి

కండిషనల్ వర్క్‌ఫ్లోలో ఉన్నట్లే షరతు ఫంక్షన్లు - అవి రూటింగ్‌ను నిర్ణయించడానికి నిర్మిత అవుట్‌పుట్‌ను పరిశీలిస్తాయి.


In [7]:
def has_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels ARE available (including priority overrides!)."""
    if not isinstance(message, AgentExecutorResponse):
        return True

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        # Check if this was a priority override
        override_indicator = " 🌟" if result.priority_override else ""

        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}{override_indicator}
            </div>
        """)
        )

        return result.has_availability
    except Exception as e:
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                <strong>⚠️  Error:</strong> {str(e)}
            </div>
        """)
        )
        return False


def no_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels are NOT available."""
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )

        return not result.has_availability
    except Exception:
        return False


print("✅ Condition functions defined")

✅ Condition functions defined


## దశ 6: కస్టమ్ డిస్ప్లే ఎగ్జిక్యూటర్ సృష్టించండి

మునుపటి ఎగ్జిక్యూటర్ మాదిరిగానే - తుది వర్క్‌ఫ్లో అవుట్‌పుట్‌ను ప్రదర్శిస్తుంది.


In [8]:
@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """Display the final result as workflow output."""
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ display_result executor created")

✅ display_result executor created


## దశ 7: పర్యావరణ వేరియబుల్స్ లోడ్ చేయండి

LLM క్లయింట్ (GitHub Models లేదా OpenAI) ను కాన్ఫిగర్ చేయండి.


In [10]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(base_url=os.environ.get(
    "GITHUB_ENDPOINT"), api_key=os.environ.get("GITHUB_TOKEN"), model_id="gpt-4o")


## దశ 8: మిడిల్‌వేర్‌తో AI ఏజెంట్లను సృష్టించండి

**ముఖ్యమైన తేడా:** availability_agent సృష్టించినప్పుడు, మేము `middleware` పారామీటర్‌ను అందిస్తాము!

ఇది priority_check_middlewareను ఏజెంట్ యొక్క ఫంక్షన్ ఆహ్వాన పైప్‌లైన్‌లో చేర్చే విధానం.


In [ ]:
# Agent 1: Check availability with tool + middleware
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), message (string), "
            "and priority_override (bool, true if priority member got special access). "
            "The message should summarize the availability status and mention if priority override occurred."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
        middleware=[priority_check_middleware],  # 🌟 MIDDLEWARE INJECTION!
    ),
    id="availability_agent",
)

# Agent 2: Suggest alternative (when no rooms)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 3: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "If priority_override is true in the input, mention that they received priority member access. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created 3 Agents:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - WITH priority_check_middleware 🌟</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
        </ul>
    </div>
""")
)

## దశ 9: వర్క్‌ఫ్లోని నిర్మించండి

మునుపటి వర్క్‌ఫ్లో నిర్మాణం మాదిరిగానే - లభ్యత ఆధారంగా షరతుల రూటింగ్.


In [12]:
# Build the workflow with conditional routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    # NO AVAILABILITY PATH
    .add_edge(availability_agent, alternative_agent, condition=no_availability_condition)
    .add_edge(alternative_agent, display_result)
    # HAS AVAILABILITY PATH (can be triggered by middleware override!)
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Conditional Routing (Middleware-Aware):</strong><br>
            • If <strong>NO availability</strong> → alternative_agent → display_result<br>
            • If <strong>availability</strong> (or 🌟 <strong>priority override</strong>) → booking_agent → display_result
        </p>
    </div>
""")
)

## దశ 10: పరీక్ష కేసు 1 - సాధారణ వినియోగదారు పారిస్‌లో (ఓవర్‌రైడ్ లేదు)

ఒక సాధారణ వినియోగదారు పారిస్‌లో గది బుక్ చేయడానికి ప్రయత్నిస్తాడు → గదులు లేవు → alternative_agent కు మారుతుంది


In [13]:
# Set as regular user
set_user("regular_user")

display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Regular User + Paris</h3>
        <p style='margin: 0;'><strong>Expected:</strong> No rooms → No middleware override → Alternative suggestion</p>
    </div>
""")
)

# Create request
request_regular = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run workflow
events_regular = await workflow.run(request_regular)
outputs_regular = events_regular.get_outputs()

# Display results
if outputs_regular:
    result_regular = AlternativeResult.model_validate_json(outputs_regular[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: #fff; border: 2px solid #ff9800; border-radius: 12px; margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #e65100;'>📊 RESULT (Regular User)</h3>
            <div style='background: #fff3e0; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                <p style='margin: 0 0 10px 0;'><strong>Middleware:</strong> No priority override (regular user)</p>
                <p style='margin: 0 0 10px 0;'><strong>Alternative:</strong> 🏨 {result_regular.alternative_destination}</p>
                <p style='margin: 0;'><strong>Reason:</strong> {result_regular.reason}</p>
            </div>
        </div>
    """)
    )

## దశ 11: పరీక్ష కేసు 2 - 🌟 ప్రాధాన్యత గల వినియోగదారు పారిస్‌లో (ఓవర్‌రైడ్‌తో!)

ఒక ప్రాధాన్యత గల సభ్యుడు పారిస్ బుక్ చేయడానికి ప్రయత్నిస్తాడు → ప్రారంభంలో గదులు లేవు → 🌟 మిడిల్‌వేర్ ఓవర్‌రైడ్ చేస్తుంది! → booking_agent కు మార్గం చూపుతుంది

**ఇది మిడిల్‌వేర్ శక్తిని ప్రదర్శించే ముఖ్యమైన ఉదాహరణ!**


In [14]:
# Set as priority user
set_user("priority_user")

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #333;'>🧪 TEST CASE 2: 🌟 Priority User + Paris</h3>
        <p style='margin: 0; color: #555;'><strong>Expected:</strong> No rooms → 🌟 MIDDLEWARE OVERRIDE → Rooms available → Booking suggestion!</p>
    </div>
""")
)

# Create request
request_priority = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run workflow
events_priority = await workflow.run(request_priority)
outputs_priority = events_priority.get_outputs()

# Display results
if outputs_priority:
    result_priority = BookingConfirmation.model_validate_json(outputs_priority[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; 
                    box-shadow: 0 8px 16px rgba(255,165,0,0.4); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 RESULT (Priority Member) 🌟</h3>
            <div style='background: white; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available (Priority Override!)</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Middleware:</strong> 🌟 OVERRIDE ACTIVATED!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_priority.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_priority.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_priority.message}</p>
                <div style='margin-top: 15px; padding: 15px; background: #fff3cd; border-radius: 6px; border-left: 4px solid #FF6B35;'>
                    <strong>💡 What Just Happened:</strong><br>
                    1. hotel_booking tool returned "no availability"<br>
                    2. priority_check_middleware intercepted the result<br>
                    3. Middleware checked user status: priority_user ✅<br>
                    4. Middleware OVERRODE the result to "available"<br>
                    5. Workflow routed to booking_agent instead of alternative_agent!
                </div>
            </div>
        </div>
    """)
    )

## దశ 12: పరీక్ష కేసు 3 - ప్రాధాన్యత వినియోగదారు స్టాక్‌హోమ్‌లో (ఇప్పటికే అందుబాటులో ఉంది)

ప్రాధాన్యత వినియోగదారు స్టాక్‌హోమ్ ప్రయత్నిస్తారు → గదులు అందుబాటులో ఉన్నాయి → ఎటువంటి ఓవర్‌రైడ్ అవసరం లేదు → booking_agent కు మార్గం

ఇది మిడిల్‌వేర్ అవసరమైనప్పుడు మాత్రమే పనిచేస్తుందని చూపిస్తుంది!


In [15]:
# Priority user is still set from previous test

display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 3: Priority User + Stockholm</h3>
        <p style='margin: 0;'><strong>Expected:</strong> Rooms available → No override needed → Booking suggestion</p>
    </div>
""")
)

# Create request
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], should_respond=True
)

# Run workflow
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; 
                    box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 RESULT (Priority User - No Override Needed)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available (Natural)</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Middleware:</strong> No override needed</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
                <div style='margin-top: 15px; padding: 15px; background: #e8f5e9; border-radius: 6px; border-left: 4px solid #4caf50;'>
                    <strong>💡 Middleware Behavior:</strong><br>
                    • hotel_booking returned "available" naturally<br>
                    • Middleware saw available = true → No override needed<br>
                    • Workflow proceeded normally to booking_agent
                </div>
            </div>
        </div>
    """)
    )

## ముఖ్యమైన విషయాలు మరియు మిడిల్‌వేర్ కాన్సెప్ట్‌లు

### ✅ మీరు నేర్చుకున్నది:

#### **1. ఫంక్షన్-ఆధారిత మిడిల్‌వేర్ ప్యాటర్న్**

మిడిల్‌వేర్ ఒక సింపుల్ అసింక్ ఫంక్షన్ ఉపయోగించి ఫంక్షన్ కాల్స్‌ను అడ్డుకుంటుంది:

```python
async def my_middleware(
    context: FunctionInvocationContext,
    next: Callable,
) -> None:
    # ఫంక్షన్ అమలు ముందు
    print("Intercepting...")
    
    # ఫంక్షన్ అమలు చేయండి
    await next(context)
    
    # ఫంక్షన్ అమలు తర్వాత - ఫలితాన్ని పరిశీలించండి
    if context.result:
        # అవసరమైతే ఫలితాన్ని మార్చండి
        context.result = modified_value
```

#### **2. కాంటెక్స్ట్ యాక్సెస్ మరియు ఫలితాన్ని ఓవర్‌రైడ్ చేయడం**

- `context.function` - కాల్ చేయబడుతున్న ఫంక్షన్‌ను యాక్సెస్ చేయండి
- `context.arguments` - ఫంక్షన్ ఆర్గుమెంట్స్‌ను చదవండి
- `context.kwargs` - అదనపు ప్యారామీటర్లను యాక్సెస్ చేయండి
- `await next(context)` - ఫంక్షన్‌ను ఎగ్జిక్యూట్ చేయండి
- `context.result` - ఫంక్షన్ అవుట్‌పుట్‌ను చదవండి/మార్చండి

#### **3. బిజినెస్ లాజిక్ అమలు**

మా మిడిల్‌వేర్ ప్రాధాన్యత సభ్యుల ప్రయోజనాలను అమలు చేస్తుంది:
- **సాధారణ యూజర్లు**: మార్పులు లేవు, స్టాండర్డ్ వర్క్‌ఫ్లో
- **ప్రాధాన్యత యూజర్లు**: "అందుబాటులో లేదు" → "అందుబాటులో ఉంది" అని ఓవర్‌రైడ్ చేస్తుంది
- **కండిషనల్ లాజిక్**: అవసరమైనప్పుడు మాత్రమే ఓవర్‌రైడ్ చేస్తుంది

#### **4. ఒకే వర్క్‌ఫ్లో, వేర్వేరు ఫలితాలు**

మిడిల్‌వేర్ శక్తి:
- ✅ వర్క్‌ఫ్లో నిర్మాణంలో మార్పులు లేవు
- ✅ టూల్ ఫంక్షన్‌లో మార్పులు లేవు
- ✅ కండిషనల్ రౌటింగ్ లాజిక్‌లో మార్పులు లేవు
- ✅ కేవలం మిడిల్‌వేర్ → వేర్వేరు ప్రవర్తన!

### 🚀 వాస్తవ ప్రపంచ అనువర్తనాలు:

1. **VIP/ప్రీమియం ఫీచర్లు**
   - ప్రీమియం యూజర్ల కోసం రేట్ లిమిట్స్‌ను ఓవర్‌రైడ్ చేయండి
   - రిసోర్సులకు ప్రాధాన్యత యాక్సెస్ అందించండి
   - ప్రీమియం ఫీచర్లను డైనమిక్‌గా అన్‌లాక్ చేయండి

2. **A/B టెస్టింగ్**
   - యూజర్లను వేర్వేరు అమలు పద్ధతులకు రూట్ చేయండి
   - నిర్దిష్ట యూజర్లతో కొత్త ఫీచర్లను టెస్ట్ చేయండి
   - ఫీచర్ రోలౌట్స్‌ను క్రమంగా అమలు చేయండి

3. **సెక్యూరిటీ & కంప్లయన్స్**
   - ఫంక్షన్ కాల్స్‌ను ఆడిట్ చేయండి
   - సెన్సిటివ్ ఆపరేషన్లను బ్లాక్ చేయండి
   - బిజినెస్ రూల్స్‌ను అమలు చేయండి

4. **పర్ఫార్మెన్స్ ఆప్టిమైజేషన్**
   - నిర్దిష్ట యూజర్ల కోసం ఫలితాలను క్యాష్ చేయండి
   - అవసరమైనప్పుడు ఖర్చుతో కూడిన ఆపరేషన్లను స్కిప్ చేయండి
   - డైనమిక్ రిసోర్స్ అలొకేషన్ చేయండి

5. **ఎర్రర్ హ్యాండ్లింగ్ & రీట్రై**
   - ఎర్రర్లను గ్రేస్‌ఫుల్‌గా క్యాచ్ చేసి హ్యాండిల్ చేయండి
   - రీట్రై లాజిక్‌ను అమలు చేయండి
   - ప్రత్యామ్నాయ అమలులకు ఫాల్‌బ్యాక్ చేయండి

6. **లాగింగ్ & మానిటరింగ్**
   - ఫంక్షన్ ఎగ్జిక్యూషన్ టైమ్స్‌ను ట్రాక్ చేయండి
   - ప్యారామీటర్లు మరియు ఫలితాలను లాగ్ చేయండి
   - యూజేజ్ ప్యాటర్న్స్‌ను మానిటర్ చేయండి

### 🔑 డెకరేటర్లతో తేడాలు:

| ఫీచర్ | డెకరేటర్ | మిడిల్‌వేర్ |
|---------|-----------|------------|
| **స్కోప్** | సింగిల్ ఫంక్షన్ | ఏజెంట్‌లోని అన్ని ఫంక్షన్లు |
| **ఫ్లెక్సిబిలిటీ** | డెఫినిషన్ వద్ద ఫిక్స్ | రన్‌టైమ్‌లో డైనమిక్ |
| **కాంటెక్స్ట్** | పరిమితం | పూర్తి ఏజెంట్ కాంటెక్స్ట్ |
| **కంపోజిషన్** | మల్టిపుల్ డెకరేటర్లు | మిడిల్‌వేర్ పైప్‌లైన్ |
| **ఏజెంట్-అవేర్** | లేదు | ఉంది (ఏజెంట్ స్టేట్ యాక్సెస్) |

### 📚 మిడిల్‌వేర్‌ను ఎప్పుడు ఉపయోగించాలి:

✅ **మిడిల్‌వేర్‌ను ఉపయోగించండి:**
- యూజర్/సెషన్ స్టేట్ ఆధారంగా ప్రవర్తనను మార్చాలి
- బహుళ ఫంక్షన్లకు లాజిక్‌ను అప్లై చేయాలి
- ఏజెంట్-లెవల్ కాంటెక్స్ట్ యాక్సెస్ అవసరం
- క్రాస్-కట్టింగ్ కాంసెర్న్స్ (లాగింగ్, ఆథ్, మొదలైనవి) అమలు చేయాలి

❌ **మిడిల్‌వేర్‌ను ఉపయోగించకండి:**
- సింపుల్ ఇన్‌పుట్ వాలిడేషన్ (Pydantic ఉపయోగించండి)
- ఫంక్షన్-స్పెసిఫిక్ లాజిక్ (ఫంక్షన్‌లో ఉంచండి)
- వన్-టైమ్ మార్పులు (ఫంక్షన్‌ను మార్చండి)

### 🎓 అడ్వాన్స్‌డ్ ప్యాటర్న్స్:

```python
# బహుళ మిడిల్‌వేర్ (నిర్వహణ క్రమం ముఖ్యం!)
middleware=[
    logging_middleware,      # మొదట లాగ్స్
    auth_middleware,         # ఆ తర్వాత ఆథ్‌ను తనిఖీ చేస్తుంది
    cache_middleware,        # ఆ తర్వాత కాష్‌ను తనిఖీ చేస్తుంది
    rate_limit_middleware,   # ఆ తర్వాత రేట్ లిమిట్స్ చేస్తుంది
    priority_check_middleware  # చివరగా ప్రాధాన్యత తనిఖీ
]

# షరతుల ఆధారంగా మిడిల్‌వేర్ నిర్వహణ
async def conditional_middleware(context, next):
    if should_execute(context):
        await next(context)
        # ఫలితాన్ని మార్చండి
    else:
        # పూర్తిగా నిర్వహణను దాటవేయండి
        context.result = cached_value
```

### 🔗 సంబంధిత కాన్సెప్ట్‌లు:

- **ఏజెంట్ మిడిల్‌వేర్**: ఏజెంట్.run() కాల్స్‌ను అడ్డుకుంటుంది
- **ఫంక్షన్ మిడిల్‌వేర్**: టూల్ ఫంక్షన్ కాల్స్‌ను అడ్డుకుంటుంది (మేము ఉపయోగించినది!)
- **మిడిల్‌వేర్ పైప్‌లైన్**: క్రమంగా అమలు అయ్యే మిడిల్‌వేర్ చైన్
- **కాంటెక్స్ట్ ప్రొపగేషన్**: మిడిల్‌వేర్ చైన్ ద్వారా స్టేట్‌ను పాస్ చేయడం


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**విమర్శ**:  
ఈ పత్రాన్ని AI అనువాద సేవ [Co-op Translator](https://github.com/Azure/co-op-translator) ఉపయోగించి అనువదించారు. మేము ఖచ్చితత్వానికి ప్రయత్నిస్తున్నప్పటికీ, ఆటోమేటెడ్ అనువాదాలలో తప్పులు లేదా అసమానతలు ఉండవచ్చు. దాని స్వదేశీ భాషలోని అసలు పత్రాన్ని అధికారం కలిగిన మూలంగా పరిగణించాలి. కీలకమైన సమాచారం కోసం, ప్రొఫెషనల్ మానవ అనువాదాన్ని సిఫారసు చేస్తాము. ఈ అనువాదాన్ని ఉపయోగించడం వల్ల కలిగే ఏవైనా అపార్థాలు లేదా తప్పుదారులు కోసం మేము బాధ్యత వహించము.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
